## Klasyfikacja gatunków irysów na podstawie ich cech charakterystycznych

W tym notatniku przeprowadzono klasyfikacji gatunków irysów, przy użyciu klasyfikatora typu Naive Bayes dostępnego w bibliotece MLlib środowiska obliczeniowego Spark

## Opis zbioru danych

Zbiór danych zawiera opis trzech gatunków irysów (setosa, versicolor, virginica) — po 50 instancji każdego gatunku. Każda instancja posiada 4 cechy:
- długość działek kielicha w centymetrach (_ang._ sepal length)
- szerokość działek kielicha w centymetrach (_ang._ sepal width)
- długość płatków w centymetrach (_ang._ petal length)
- szerokość płatków w centymetrach (_ang._ petal width)

Ponadto każda instancji przypisano odpowiadający jej gatunek

_[Zbiór danych został zaczerpnięty z repozytorium UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/iris)_

### Importowanie potrzebnych bibliotek

In [26]:
import findspark
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession

### Inicjowanie sesji

In [27]:
findspark.init()
spark = SparkSession.builder.getOrCreate()

### Importowanie zbioru danych

Wyświetlenie fragmentu zbioru danych

In [28]:
irisDS = spark.read.format("csv").option("header", "true").option("delimiter", ",").option("quote", "").option("inferSchema", "true").load("/home/jovyan/work/iris.csv")
irisDS.show()

+-----------+----------+-----------+----------+-----------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|    Species|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|
|        5.4|       3.7|        1.5|       0.2|Iris-setosa|
|        4.8|       3.4|        1.6|       0.2|Iris-setosa|
|        4.8|       3.0|        1.4|       0.1|Iris-setosa|
|        4.3|       3.0|        1.1|    

### Sprawdzanie, czy spark poprawnie rozpoznał poszczególne kolumny

Wszystkie pola poza polem opisującym gatunki (_ang._ Species) powinny zawierać dane numeryczne

In [29]:
irisDS.printSchema()

root
 |-- SepalLength: double (nullable = true)
 |-- SepalWidth: double (nullable = true)
 |-- PetalLength: double (nullable = true)
 |-- PetalWidth: double (nullable = true)
 |-- Species: string (nullable = true)



### Tworzenie obiektów klas StringIndexer oraz VectorAssembler

Obiekty te potrzebne są odpowiedniego przygotowania danych dla klasyfikatora. Klasyfikator przyjmuje dane w postaci dwukolumnowej tabeli. Pierwsza kolumna nosi nazwę _features_, zawiera ona wektor cech charakterystycznych. Natomiast druga kolumna nazywa się label i przechowuje numery kategorii. Obiekt klasy _StringIndexer_ służy do zamiany nazw gatunków badanych irysów na numeryczne etykiety. Natomiast obiekt klasy VectorAssembler służy do złączenia poszczególnych kolumn z cechami charakterystycznymi w jeden wektor.

In [30]:
labelIndexer = StringIndexer(inputCol="Species", outputCol="label")

vecAssembler = VectorAssembler(inputCols=["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"], outputCol="features")

### Podział zbioru danych na zbiór treningowy i testowy

Zbiór danych zostanie losowo podzielony na dwa zbiory. Zbiór treningowy będzie zawierał 70% danych z początkowego zbioru. Zostanie on wykorzystany do zbudowania modelu klasyfikującego. Zbiór testowy będzie zawierał 30% danych z początkowego zbioru. Zostanie on wykorzystany do ewaluacji zbudowanego modelu. Po podziale wypisywana jest ilość instancji znajdujących się w każdym ze zbiorów.

In [31]:
(trainingData, testData) = irisDS.randomSplit([0.7, 0.3], seed=100)

print(f"Ilość instancji w zbiorze treningowym: {trainingData.count()}")
print(f"Ilość instancji w zbiorze testowym: {testData.count()}")

Ilość instancji w zbiorze treningowym: 104
Ilość instancji w zbiorze testowym: 46


### Tworzenie i trenowanie wieloklasowego klasyfikatora przy użyciu algorytmu Naive Bayes

Jak wspominano na początku tego notatnika, klasyfikator ten spróbuje przewidzieć jakiego gatunku jest dana instancja irysa, na podstawie jej cech charakterystycznych. __Należy pamiętać, że ten algorytm _przyjmuje_, że podane mu cechu są od siebie niezależne i nieujemne__. Przy tworzeniu modelu zostanie wykorzystany potok z biblioteki MLlib środowiska Spark.

In [32]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

pipeline = Pipeline(stages=[labelIndexer, vecAssembler, nb])

model = pipeline.fit(trainingData)

### Uruchomienie zbudowanego modelu na danych testowych

Na podstawie zdobytej wiedzy, model przewiduje gatunek instancji irysów ze zbioru testowego.

In [33]:
predictions = model.transform(testData)

### Wyświetlanie budowy danych wynikowych



In [34]:
predictions.printSchema()

root
 |-- SepalLength: double (nullable = true)
 |-- SepalWidth: double (nullable = true)
 |-- PetalLength: double (nullable = true)
 |-- PetalWidth: double (nullable = true)
 |-- Species: string (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



### Prezentacja przewidywań

W kolumnie _label_ znajduje się faktyczny numer kategorii. W kolumnie _prediction_ znajduje się przewidywany numer kategorii (nadany instancji przez klasyfikator). W kolumnie _probability_ znajduje się wektor z prawdopodobieństwami, z jakimi dana instancja należy do każdej z kategorii.

In [35]:
predictions.select("label", "prediction", "probability").show(50, truncate=False)

+-----+----------+-------------------------------------------------------------+
|label|prediction|probability                                                  |
+-----+----------+-------------------------------------------------------------+
|2.0  |2.0       |[0.20253050754778992,0.11205481371745843,0.6854146787347515] |
|2.0  |2.0       |[0.22610513002031704,0.12516423426180695,0.6487306357178761] |
|2.0  |2.0       |[0.20155443460425587,0.10855263459197305,0.6898929308037711] |
|2.0  |2.0       |[0.20855321122455778,0.11231546581200702,0.6791313229634351] |
|2.0  |2.0       |[0.2398081472107575,0.13230333720250126,0.6278885155867413]  |
|2.0  |2.0       |[0.2018436957183859,0.10766518303273141,0.6904911212488827]  |
|2.0  |2.0       |[0.2018436957183859,0.10766518303273141,0.6904911212488827]  |
|2.0  |2.0       |[0.17954001348424262,0.0939934849813415,0.7264665015344159]  |
|2.0  |2.0       |[0.18366447593700436,0.09631770152992547,0.7200178225330701] |
|2.0  |2.0       |[0.1917821

### Ewaluacja modelu

Do ewaluacji badanego modelu wykorzystano ewaluator typu _MulticlassClassificationEvaluator_ z biblioteki MLlib środowiska Spark. Posłużył on do obliczenia i wyświetlenia dokładności zbudowanego modelu.

In [36]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Dokładność modelu wynosi około {round(accuracy * 100, 2)}%")

Dokładność modelu wynosi około 91.3%


### Zatrzymanie sesji

In [37]:
spark.stop()